<a href="https://colab.research.google.com/github/rodrigourech/ivi/blob/main/data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Get data
!apt-get install -y lftp

# Lade fehlende ZIP-Dateien vom FTP-Server (tas & pr, RCP45 + RCP85)
!lftp -u seifert_FHNW,'iB7843(F' -e "set ftp:ssl-force true; set ssl:verify-certificate no; \
cd QMstations/tas; get CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip; \
cd ../pr; get CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip; \
get CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip; \
quit" ftp://climftp.cscs.ch

# Entpacke alle ZIP-Dateien in ./data/
!mkdir -p data
!unzip -o CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip -d ./data/
!unzip -o CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip -d ./data/
!unzip -o CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip -d ./data/
!unzip -o CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip -d ./data/


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lftp is already the newest version (4.9.2-1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
cd ok, cwd=/QMstations/tas
get: /content/CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip: file already exists and xfer:clobber is unset
cd ok, cwd=/QMstations/pr
get: /content/CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip: file already exists and xfer:clobber is unset
get: /content/CH2018_pr_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099_csv.zip: file already exists and xfer:clobber is unset
unzip:  cannot find or open CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip, CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip.zip or CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP45_QMstations_1981-2099_csv.zip.ZIP.
Archive:  CH2018_tas_CLMCOM-CCLM4_ECEARTH_EUR11_RCP85_QMstations_1981-2099

In [3]:
#  Daten kombinieren mit pandas
import pandas as pd
import os

# Parameter definieren
stations = {
    "BER": "Bern",
    "ZER": "Zürich",
    "LUG": "Lugano",
    "DAV": "Davos"
}
variables = ["tas", "pr"]
scenarios = ["RCP45", "RCP85"]
model = "CLMCOM-CCLM4_ECEARTH_EUR11"

# Leerer DataFrame zur Kombination
df_all = pd.DataFrame()

# Schleife durch alle Kombinationen
for var in variables:
    for rcp in scenarios:
        base = f"CH2018_{var}_{model}_{rcp}_QMstations_1981-2099"
        for code, name in stations.items():
            file = f"data/{base}_{code}.csv"
            if os.path.exists(file):
                df = pd.read_csv(file, delimiter=";", skiprows=16, parse_dates=["DATE"])
                df.columns = ["date", "value"]
                df["station"] = name
                df["station_code"] = code
                df["variable"] = var
                df["scenario"] = rcp
                df["model"] = model
                df_all = pd.concat([df_all, df], ignore_index=True)


# Exportiere kombinierten Datensatz als CSV
df_all.to_csv("df_ch2018_combined.csv", index=False)